### Test Task Solution
At first, one needs to create a database and store all the tables required for completing the technical task.

In [10]:
# Importing necessary libraries
import pandas as pd
import sqlite3

# Reading CSV files as pandas DataFrame
df_1 = pd.read_csv('ACTIVE_CLIENTS.csv', index_col='CLIENT_ID')
df_2 = pd.read_csv('OFFERS.csv', index_col='OFFER_ID')
df_3 = pd.read_csv('CARDS.csv', index_col='CLIENT_ID')
df_4_1 = pd.read_csv('BONUS.csv', index_col='CLIENT_ID')
df_4_2 = pd.read_csv('MCC_CATEGORIES.csv', index_col='MCC_CODE')

In [2]:
# Connecting/creating 'DataBase.db'
db = sqlite3.connect("DataBase.db")

In [3]:
# Storing created tables in the 'DataBase.db' database
df_1.to_sql('ACTIVE_CLIENTS', db)
df_2.to_sql('OFFERS', db)
df_3.to_sql('CARDS', db)
df_4_1.to_sql('BONUS', db)
df_4_2.to_sql('MCC_CATEGORIES', db)

3

Now, one can begin using SQL in the Jupyter Notebook by loading the SQL extension and connecting to our database.

In [11]:
# Loading the SQL extension
%load_ext sql

# Connecting to the 'DataBase.db'
%sql sqlite:///DataBase.db

#### 1. The first task:

<b>Data Description</b>: The ACTIVE_CLIENTS table contains a monthly snapshot of bank clients who made any transactions. Attributes: reporting month (report_month) and client identifier (client_id). We consider a client to have "churned" from the bank in month N if they are active (present in the ACTIVE_CLIENTS table) in month N and inactive in months N+1, N+2, N+3.

| REPORT_MONTH | CLIENT_ID  |
|--------------|------------|
| 2018-01-01   | 1847982357 |
| 2018-01-01   | 938475     |
| 2018-02-01   | 1847982357 |
| 2018-02-01   | 6789998   |
| 2018-03-01   | 67900001  |
| ...          | ...        |

<b>Task</b>: Display the number of active clients for each month; display the proportion of clients who "churned" in each month.

First, let's examine the table required to complete the first task.

In [7]:
%%sql 
SELECT * 
FROM ACTIVE_CLIENTS
ORDER BY CLIENT_ID;

 * sqlite:///DataBase.db
Done.


CLIENT_ID,REPORT_MONTH
300000,2018-08-09
300000,2019-01-09
938475,2018-01-01
938475,2018-02-01
1000000,2018-06-06
1000000,2018-10-06
2000000,2018-07-08
2000000,2018-10-08
6789998,2018-01-01
6789998,2018-02-01


It is not essential to track the number of actions performed by each client within a month, so we can consolidate the data by ensuring that there is no more than one unique CLIENT_ID in a given month.

In [8]:
%%sql
SELECT DISTINCT CLIENT_ID, DATE(REPORT_MONTH, 'start of month') AS REPORT_MONTH
FROM ACTIVE_CLIENTS;

 * sqlite:///DataBase.db
Done.


CLIENT_ID,REPORT_MONTH
300000,2018-08-01
300000,2019-01-01
938475,2018-01-01
938475,2018-02-01
1000000,2018-06-01
1000000,2018-10-01
2000000,2018-07-01
2000000,2018-10-01
6789998,2018-01-01
6789998,2018-02-01


Next, we can create an additional column indicating the next active month immediately following the current active month, calculate the month difference taking into account years and assign if the month difference is less than or equal to 3, than the client is active in the current month, otherwise it is churned.

In [9]:
%%sql
WITH SimplifiedData AS (
    SELECT DISTINCT CLIENT_ID, DATE(REPORT_MONTH, 'start of month') AS REPORT_MONTH
    FROM ACTIVE_CLIENTS
)
SELECT
    CLIENT_ID,
    REPORT_MONTH,
    LEAD(REPORT_MONTH, 1) OVER (PARTITION BY CLIENT_ID ORDER BY REPORT_MONTH) AS LEAD_REPORT_MONTH,
    (
        (LEAD(strftime('%Y', REPORT_MONTH), 1) OVER (PARTITION BY CLIENT_ID ORDER BY REPORT_MONTH) - strftime('%Y', REPORT_MONTH)) * 12 +
        (LEAD(strftime('%m', REPORT_MONTH), 1) OVER (PARTITION BY CLIENT_ID ORDER BY REPORT_MONTH) - strftime('%m', REPORT_MONTH))
    ) AS MONTH_DIFFERENCE,
    CASE
        WHEN (
            (
                (LEAD(strftime('%Y', REPORT_MONTH), 1) OVER (PARTITION BY CLIENT_ID ORDER BY REPORT_MONTH) - strftime('%Y', REPORT_MONTH)) * 12 +
                (LEAD(strftime('%m', REPORT_MONTH), 1) OVER (PARTITION BY CLIENT_ID ORDER BY REPORT_MONTH) - strftime('%m', REPORT_MONTH))
            ) <= 3
        ) THEN 'active'
        ELSE 'churned'
    END AS STATUS
FROM SimplifiedData
order by REPORT_MONTH;

 * sqlite:///DataBase.db
Done.


CLIENT_ID,REPORT_MONTH,LEAD_REPORT_MONTH,MONTH_DIFFERENCE,STATUS
938475,2018-01-01,2018-02-01,1,active
6789998,2018-01-01,2018-02-01,1,active
67900001,2018-01-01,2018-02-01,1,active
1847982357,2018-01-01,2018-02-01,1,active
938475,2018-02-01,None,None,churned
6789998,2018-02-01,2018-03-01,1,active
67900001,2018-02-01,2018-03-01,1,active
1847982357,2018-02-01,2018-03-01,1,active
6789998,2018-03-01,2018-04-01,1,active
67900001,2018-03-01,2018-04-01,1,active


The last step is to group by 'REPORT_MONTH' and exclude the last 3 months because we cannot determine if the clients will remain active in the next 3 months.

In [29]:
%%sql
WITH SimplifiedData AS (
    SELECT
        CLIENT_ID,
        REPORT_MONTH,
        CASE
            WHEN (
                (
                    (LEAD(strftime('%Y', REPORT_MONTH), 1) OVER (PARTITION BY CLIENT_ID ORDER BY REPORT_MONTH) - strftime('%Y', REPORT_MONTH)) * 12 +
                    (LEAD(strftime('%m', REPORT_MONTH), 1) OVER (PARTITION BY CLIENT_ID ORDER BY REPORT_MONTH) - strftime('%m', REPORT_MONTH))
                ) <= 3
            ) THEN 'active'
            ELSE 'churned'
        END AS STATUS
    FROM (
        SELECT DISTINCT CLIENT_ID, DATE(REPORT_MONTH, 'start of month') AS REPORT_MONTH
        FROM ACTIVE_CLIENTS
    )
)
SELECT
    REPORT_MONTH,
    STATUS,
    COUNT(*) AS ACT_CLIENT_NUM
FROM
    SimplifiedData
WHERE
    STATUS = 'active'
    AND REPORT_MONTH <= date('now', '-3 months')
GROUP BY
    REPORT_MONTH;

 * sqlite:///DataBase.db
Done.


REPORT_MONTH,STATUS,ACT_CLIENT_NUM
2018-01-01,active,4
2018-02-01,active,3
2018-03-01,active,2
2018-04-01,active,1
2018-07-01,active,1
2018-09-01,active,1
2018-11-01,active,1


In [28]:
%%sql
WITH SimplifiedData AS (
    SELECT
        CLIENT_ID,
        REPORT_MONTH,
        CASE
            WHEN (
                (
                    (LEAD(strftime('%Y', REPORT_MONTH), 1) OVER (PARTITION BY CLIENT_ID ORDER BY REPORT_MONTH) - strftime('%Y', REPORT_MONTH)) * 12 +
                    (LEAD(strftime('%m', REPORT_MONTH), 1) OVER (PARTITION BY CLIENT_ID ORDER BY REPORT_MONTH) - strftime('%m', REPORT_MONTH))
                ) <= 3
            ) THEN 'active'
            ELSE 'churned'
        END AS STATUS
    FROM (
        SELECT DISTINCT CLIENT_ID, DATE(REPORT_MONTH, 'start of month') AS REPORT_MONTH
        FROM ACTIVE_CLIENTS
    )
)
SELECT
    REPORT_MONTH,
    SUM(CASE WHEN STATUS = 'churned' THEN 1 ELSE 0 END) * 1.0 / COUNT(*) AS CHUR_FRACTION
FROM
    SimplifiedData
WHERE
    REPORT_MONTH <= date('now', '-3 months')
GROUP BY
    REPORT_MONTH;

 * sqlite:///DataBase.db
Done.


REPORT_MONTH,CHUR_FRACTION
2018-01-01,0.0
2018-02-01,0.25
2018-03-01,0.3333333333333333
2018-04-01,0.5
2018-05-01,1.0
2018-06-01,1.0
2018-07-01,0.0
2018-08-01,1.0
2018-09-01,0.0
2018-10-01,1.0


Thus, the first table displays the count of active clients for each month, while the second one indicates the fraction of customers who churned each month.

#### 2. The second task:

<b>Data Description</b>: The OFFERS table contains information about credit offers for bank customers. Each offer has an identifier (offer_id), a start date (offer_start_date), and an expiration date (offer_expiration_date).

<b>Task</b>: Display the number of active offers for each day in the year 2018.

| OFFER_ID | OFFER_START_DATE | OFFER_EXPIRATION_DATE |
|----------|------------------|-----------------------|
|   83942  |  2017-12-01      |  2018-02-01           |
|   94859  |  2018-05-03      |  2018-10-19           |
|   ...    |  ...             |  ...                  |


Below, we can see the table used for this task.

In [12]:
%%sql 
SELECT * 
FROM OFFERS;

 * sqlite:///DataBase.db
Done.


OFFER_ID,OFFER_START_DATE,OFFER_EXPIRATION_DATE
83942,2017-12-01,2018-02-01
94858,2018-03-03,2018-07-19
94859,2018-05-03,2018-10-19


First, we can create a temporary table with all the dates of 2018.

In [13]:
%%sql
WITH Dates AS (
    SELECT DATE('2018-01-01') AS DATE
    UNION ALL
    SELECT DATE(DATE, '+1 day') 
    FROM Dates
    WHERE DATE < '2018-12-31'
)
SELECT *
FROM Dates

 * sqlite:///DataBase.db
Done.


DATE
2018-01-01
2018-01-02
2018-01-03
2018-01-04
2018-01-05
2018-01-06
2018-01-07
2018-01-08
2018-01-09
2018-01-10


Next, we can create another table that joins the Dates table with the OFFERS table and assigns an 'OFFER_ID' if the date falls within the offer period (between OFFER_START_DATE and OFFER_EXPIRATION_DATE) for each 'OFFER_ID'.

In [14]:
%%sql
WITH Dates AS (
    SELECT DATE('2018-01-01') AS DATE
    UNION ALL
    SELECT DATE(DATE, '+1 day') 
    FROM Dates
    WHERE DATE < '2018-12-31'
)
SELECT D.DATE, 
       O.OFFER_ID
FROM Dates D
LEFT JOIN OFFERS O 
    ON D.DATE BETWEEN O.OFFER_START_DATE AND O.OFFER_EXPIRATION_DATE;

 * sqlite:///DataBase.db
Done.


DATE,OFFER_ID
2018-01-01,83942
2018-01-02,83942
2018-01-03,83942
2018-01-04,83942
2018-01-05,83942
2018-01-06,83942
2018-01-07,83942
2018-01-08,83942
2018-01-09,83942
2018-01-10,83942


Finally, we can query the resulting table (OfferCounts) to get the count of 'OFFER_ID' for each date.

In [18]:
%%sql
WITH Dates AS (
    SELECT DATE('2018-01-01') AS DATE
    UNION ALL
    SELECT DATE(DATE, '+1 day') 
    FROM Dates
    WHERE DATE < '2018-12-31'
)
SELECT D.DATE,
       COUNT(O.OFFER_ID) AS ACTIVE_OFFER_COUNT
FROM Dates D
LEFT JOIN OFFERS O 
    ON D.DATE BETWEEN O.OFFER_START_DATE AND O.OFFER_EXPIRATION_DATE
GROUP BY D.DATE;

 * sqlite:///DataBase.db
Done.


DATE,ACTIVE_OFFER_COUNT
2018-01-01,1
2018-01-02,1
2018-01-03,1
2018-01-04,1
2018-01-05,1
2018-01-06,1
2018-01-07,1
2018-01-08,1
2018-01-09,1
2018-01-10,1


#### 3. The third task

<b>Data Description</b>: The CARDS table contains information about all the bank's customers' cards. Attributes include customer identifier (client_id), card identifier (card_id), card issue date (open_date), card closure date (close_date), and card type - debit (DC) or credit (CC) (card_type).

<b>Task</b>: Select the debit card of a customer, among all working debit cards on the date 2018-09-01, which was issued last. Output format: client_id, card_id.

| CLIENT_ID | CARD_ID     | OPEN_DATE  | CLOSE_DATE | CARD_TYPE |
|-----------|------------|------------|------------|-----------|
| 1232110   | 49582985729 | 2019-01-12 | NULL       | DC        |
| 234235    | 48574092749 | 2017-03-29 | 2018-09-01 | CC        |
| ...       | ...        | ...        | ...        | ...       |

Let's first look at our initial table.

In [5]:
%%sql 
SELECT * 
FROM CARDS

 * sqlite:///DataBase.db
Done.


CLIENT_ID,CARD_ID,OPEN_DATE,CLOSE_DATE,CARD_TYPE
1232110,49582985729,2019-01-12,None,DC
234235,48574092749,2017-03-29,2018-09-01,DC
234235,485742749,2017-10-29,2018-01-01,DC
234235,43,2017-10-29,2019-03-03,DC
234235,4857409279,2017-03-29,2018-09-01,CC
234235,48574274,2017-10-29,2019-09-01,CC
234235,4857427,2017-10-29,2019-03-03,CC
1232110,495885729,2018-01-12,2018-09-01,DC
1232110,49,2017-01-12,2019-01-01,DC
1232110,4958572,2018-01-11,2019-01-02,DC


Then, cut off irrelevant data.

In [11]:
%%sql
SELECT *
FROM CARDS
WHERE CARD_TYPE = 'DC'
    AND OPEN_DATE <= '2018-09-01' 
    AND (CLOSE_DATE IS NULL OR CLOSE_DATE > '2018-09-01')
ORDER BY OPEN_DATE DESC;

 * sqlite:///DataBase.db
Done.


CLIENT_ID,CARD_ID,OPEN_DATE,CLOSE_DATE,CARD_TYPE
1232110,4958729,2018-01-12,2019-10-03,DC
1232110,4958572,2018-01-11,2019-01-02,DC
234235,43,2017-10-29,2019-03-03,DC
234235,4854274,2017-07-29,2019-01-03,DC
234235,484274,2017-03-29,2020-01-03,DC
1232110,49,2017-01-12,2019-01-01,DC


And finally, group it by 'CLIENT_ID' having the maximum 'OPEN_DATE'.

In [25]:
%%sql
SELECT CLIENT_ID, CARD_ID
FROM CARDS
WHERE CARD_TYPE = 'DC'
    AND OPEN_DATE <= '2018-09-01' 
    AND (CLOSE_DATE IS NULL OR CLOSE_DATE > '2018-09-01')
GROUP BY CLIENT_ID
HAVING OPEN_DATE = MAX(OPEN_DATE)

 * sqlite:///DataBase.db
Done.


CLIENT_ID,CARD_ID
234235,43
1232110,4958729


#### 4. The fourth task

<b>Description of the data</b>: The BONUS table contains information about bonus points awarded for purchases under the loyalty program. Attributes: client identifier (client_id), date of bonus accrual (bonus_date), the amount of accrued bonuses (bonus_cnt), the MCC code of the transaction for which the bonuses were accrued (mcc_code). The MCC_CATEGORIES table is a reference of MCC codes. Attributes: MCC code (mcc_code), category (e.g., supermarkets, transportation, pharmacies, etc., mcc_category). 

<b>Task</b>: Indicate 1000 clients who were the first to accumulate 1000 bonus points for purchases in the "Airline Tickets" and "Hotels" categories.

CLIENT_ID | BONUS_DATE | BONUS_CNT | MCC_CODE
--------- | ---------- | --------- | --------
1232110   | 2018-01-01 | 12        | 3617
234235    | 2018-06-17 | 5         | 5931
...       | ...        | ...       | ...

MCC_CODE | MCC_CATEGORY
-------- | -------------
3031     | Авиабилеты
5735     | Музыка
...      | ...


First, let's join 2 tables.

In [7]:
%%sql
SELECT *
FROM
    BONUS AS b
JOIN
    MCC_CATEGORIES AS m
ON
    b.mcc_code = m.mcc_code

 * sqlite:///DataBase.db
Done.


CLIENT_ID,BONUS_DATE,BONUS_CNT,MCC_CODE,MCC_CODE_1,MCC_CATEGORY
1,2018-01-01,12,1,1,Avia
2,2018-06-17,5,2,2,Music
3,2018-06-18,15,3,3,Hotels
4,2018-01-04,6,1,1,Avia
5,2018-09-11,34,2,2,Music
6,2018-01-10,26,3,3,Hotels
7,2018-07-17,74,1,1,Avia
8,2018-09-18,32,3,3,Hotels
9,2018-02-04,12,1,1,Avia
10,2018-12-11,43,3,3,Hotels


And keep only 'Avia' and 'Hotels' categories.

In [15]:
%%sql
SELECT b.CLIENT_ID, b.BONUS_DATE, b.BONUS_CNT, m.MCC_CATEGORY
FROM BONUS AS b
JOIN MCC_CATEGORIES AS m
    ON b.MCC_CODE = m.MCC_CODE
WHERE m.MCC_CATEGORY IN ('Avia', 'Hotels')
ORDER BY b.BONUS_DATE

 * sqlite:///DataBase.db
Done.


CLIENT_ID,BONUS_DATE,BONUS_CNT,MCC_CATEGORY
1,2018-01-01,12,Avia
1,2018-01-01,12,Avia
1,2018-01-01,12,Avia
4,2018-01-04,6,Avia
6,2018-01-10,26,Hotels
6,2018-01-11,26,Hotels
4,2018-01-14,46,Avia
1,2018-01-18,12,Avia
6,2018-01-19,26,Hotels
6,2018-01-19,26,Hotels


Then, sum bonuses for each CLIENT_ID over BONUS_DATE

In [24]:
%%sql
SELECT
    CLIENT_ID,
    BONUS_DATE,
    BONUS_CNT,
    MCC_CATEGORY,
    SUM(BONUS_CNT) OVER (PARTITION BY CLIENT_ID ORDER BY BONUS_DATE) AS BON_SUM
FROM (
    SELECT b.CLIENT_ID, b.BONUS_DATE, b.BONUS_CNT, m.MCC_CATEGORY
    FROM BONUS AS b
    JOIN MCC_CATEGORIES AS m
        ON b.MCC_CODE = m.MCC_CODE
    WHERE m.MCC_CATEGORY IN ('Avia', 'Hotels')
    ORDER BY b.BONUS_DATE
    )

 * sqlite:///DataBase.db
Done.


CLIENT_ID,BONUS_DATE,BONUS_CNT,MCC_CATEGORY,BON_SUM
1,2018-01-01,12,Avia,36
1,2018-01-01,12,Avia,36
1,2018-01-01,12,Avia,36
1,2018-01-18,12,Avia,48
2,2018-06-01,25,Avia,70
2,2018-06-01,45,Avia,70
2,2018-06-21,45,Avia,115
3,2018-06-01,15,Hotels,30
3,2018-06-01,15,Hotels,30
3,2018-06-18,15,Hotels,45


The last step is to keep data with summarized bonuses >= 50, grouping by CLIENT_ID, ordering by DATE, and Keeping top 5 values.

PS: Here, there is an example of 50 bonuses and top 5 clients for simplification, but a similar code can be used for 1000 bonuses and top 1000 clients.

In [17]:
%%sql
SELECT
    CLIENT_ID
FROM (
    SELECT
        CLIENT_ID,
        BONUS_DATE,
        BONUS_CNT,
        MCC_CATEGORY,
        SUM(BONUS_CNT) OVER (PARTITION BY CLIENT_ID ORDER BY BONUS_DATE) AS bon_sum
    FROM (
        SELECT b.CLIENT_ID, b.BONUS_DATE, b.BONUS_CNT, m.MCC_CATEGORY
        FROM BONUS AS b
        JOIN MCC_CATEGORIES AS m
            ON b.MCC_CODE = m.MCC_CODE
        WHERE m.MCC_CATEGORY IN ('Avia', 'Hotels')
        ORDER BY b.BONUS_DATE
)
)
WHERE BON_SUM >= 50
GROUP BY CLIENT_ID
HAVING BONUS_DATE = MIN(BONUS_DATE)
ORDER BY BONUS_DATE
LIMIT 5

 * sqlite:///DataBase.db
Done.


CLIENT_ID
6
4
10
9
2


#### 5. The fifth task
The sum(X) command sums all the values of column X, how do you multiply all the values of one column in SQL?

In [51]:
%%sql
SELECT MCC_CODE
FROM MCC_CATEGORIES

 * sqlite:///DataBase.db
Done.


MCC_CODE
1
2
3


In [50]:
%%sql
SELECT SUM(MCC_CODE)
FROM MCC_CATEGORIES

 * sqlite:///DataBase.db
Done.


SUM(MCC_CODE)
6


In [70]:
%%sql
SELECT EXP(SUM(LN(MCC_CODE)))
FROM your_table;

 * sqlite:///DataBase.db
(sqlite3.OperationalError) no such table: your_table
[SQL: SELECT EXP(SUM(LN(MCC_CODE)))
FROM your_table;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
